#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import when, count, isnull, expr, current_date, col, date_format, year, month, lag
from pyspark.sql.window import Window

In [0]:
# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true") \
                                  .option("header", "true") \
                                  .csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 1490878 bytes.
Wrote 3862387 bytes.
Wrote 57561 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[79]: [('codice_nuts_1', 'string'),
 ('descrizione_nuts_1', 'string'),
 ('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('sigla_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_genere_maschile', 'int'),
 ('totale_genere_femminile', 'int'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_1', 'descrizione_nuts_1', 'codice_nuts_2', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+------------------+-------------+---------------------+-------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|
+-------------+------------------+-------------+---------------------+-------------+
 ITI| Centro| ITI4| Lazio| LAZ|
 ITI| Centro| ITI3| Marche| MAR|
 ITI| Centro| ITI2| Umbria| UMB|
 ITI| Centro| ITI1| Toscana| TOS|
 ITG| Isole| ITG1| Sicilia| SIC|
 ITG| Isole| ITG2| Sardegna| SAR|
 ITH| Nord-Est| ITH5| Emilia-Romagna| EMR|
 ITH| Nord-Est| ITH1| Bolzano| BOL|
 ITH| Nord-Est| ITH4| Friuli Venezia Gi...| FVG|
 ITH| Nord-Est| ITH2| Trento| TRE|
 ITH| Nord-Est| ITH3| Veneto| VEN|
 ITC| Nord-Ovest| ITC1| Piemonte| PIE|
 ITC| Nord-Ovest| ITC2| Valle d'Aosta| VDA|
 ITC| Nord-Ovest| ITC3| Liguria| LIG|
 ITC| Nord-Ovest| ITC4| Lombardia| LOM|
 ITF| Sud| ITF1| Abruzzo| ABR|
 ITF| Sud| ITF2| Molise| MOL|
 ITF| Sud| ITF3| Campania| CAM|
 ITF| Sud| ITF5| Basilicata| BAS|
 ITF| Sud| ITF4| Puglia| PUG|
 ITF| Sud| ITF6| Calabria| CAL|
+-------------+------------------+-------------+---------------------+-------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe(['totale_genere_maschile', 'totale_genere_femminile', 'totale_generale']) \
                    .show()

+-------+----------------------+-----------------------+-----------------+
summary|totale_genere_maschile|totale_genere_femminile| totale_generale|
+-------+----------------------+-----------------------+-----------------+
 count| 210| 210| 210|
 mean| 138333.79047619048| 145673.29523809525|284007.0857142857|
 stddev| 151728.9786894938| 152578.20019731886|303588.4857882117|
 min| 457| 1307| 1764|
 max| 795710| 796399| 1592109|
+-------+----------------------+-----------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| null| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| null| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| null| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| null| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| null| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| null| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| null| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| null| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| null| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| null| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[85]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('isolamento_domiciliare', 'int'),
 ('totale_positivi', 'int'),
 ('variazione_totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('dimessi_guariti', 'int'),
 ('deceduti', 'int'),
 ('totale_casi', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_1', 'string'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1') \
                    .show(50)

Il numero di regioni è: 21

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 null| null| Lazio|
 null| null| Sicilia|
 null| null| Emilia-Romagna|
 null| null| Toscana|
 null| null| Puglia|
 null| null| Abruzzo|
 null| null| Sardegna|
 null| null| Umbria|
 null| null| Piemonte|
 null| null| P.A. Bolzano|
 null| null| Molise|
 null| null| Calabria|
 null| null| Friuli Venezia Gi...|
 null| null| Valle d'Aosta|
 null| null| P.A. Trento|
 null| null| Marche|
 null| null| Basilicata|
 null| null| Veneto|
 null| null| Campania|
 null| null| Lombardia|
 null| null| Liguria|
 1| null| P.A. Bolzano|
 172410| ITI| Umbria|
 500340| ITH| P.A. Bolzano|
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF1| Abruzzo|
 ITF| ITF3| Campania|
 ITF| ITF4| Puglia|
 ITG| ITG1| Sicilia|
 ITG| ITG2| Sardegna|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH2| P.A. Trento|
 ITH| ITH1| P.A. Bolzano|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1', 'codice_nuts_2')

data = popolazione_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1').show(num_regioni)

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'dimessi_guariti']) \
                  .show()

+-------+----------------------+------------------+--------------------+-----------------+
summary|ricoverati_con_sintomi| terapia_intensiva|totale_ospedalizzati| dimessi_guariti|
+-------+----------------------+------------------+--------------------+-----------------+
 count| 9009| 9009| 9009| 9009|
 mean| 710.998778998779| 81.4092574092574| 792.4080364080364|41151.50260850261|
 stddev| 1291.9773851854593|143.01963701043422| 1430.2559796033654|85453.85609768223|
 min| 0| 0| 0| 0|
 max| 12077| 1381| 13328| 711865|
+-------+----------------------+------------------+--------------------+-----------------+

In [0]:
dati_covid_regioni.describe(['isolamento_domiciliare', 'totale_positivi', 'variazione_totale_positivi', 'nuovi_positivi', 'deceduti', 'totale_casi', 'tamponi']) \
                  .show()

+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+
summary|isolamento_domiciliare| totale_positivi|variazione_totale_positivi| nuovi_positivi| deceduti| totale_casi| tamponi|
+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+
 count| 9009| 9009| 9009| 9009| 9009| 9009| 9009|
 mean| 11617.36285936286|12409.770895770896| 49.72005772005772|441.25163725163725|2447.3292263292265|56008.604728604725| 835210.9781329781|
 stddev| 21218.46558981881|22222.370120621243| 827.4388659103472| 855.0707239751421|4665.9360193025495|105042.01852045507|1316900.6291977828|
 min| 0| 0| -21926| -229| 0| 0| 0|
 max| 155066| 164406| 10263| 11489| 32742| 797658| 9313405|
+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 Piemonte|2020-02-27 18:00:00| 2| 0| 2| 0| 2| -1| -1| 0| 0| 2| 156| ITC| ITC1|
 Liguria|2020-03-01 17:00:00| 12| 1| 13| 8| 21| -17| -17| 4| 0| 25| 121| ITC| ITC3|
 Liguria|2020-03-02 18:00:00| 12| 1| 13| 5| 18| -3| -3| 4| 0| 22| 121| ITC| ITC3|
 Sicilia|2020-03-02 18:00:00| 2| 0| 2| 3| 5| -2| -2| 2| 0| 7| 307| ITG| ITG1|
 Piemonte|2020-03-09 18:00:00| 222| 50| 272| 65| 337| -18| -10| 0| 13| 350| 1681| ITC| ITC1|
 Calabria|2020-04-17 17:00:00| 154| 7| 161| 658| 819| -28| -18| 99| 73| 991| 21657| ITF| ITF6|
 Sardegna|2020-05-04 17:00:00| 91| 9| 100| 553| 653| -36| -2| 545| 119| 1317| 28052| ITG| ITG2|
 Basilicata|2020-05-07 17:00:00| 48| 2| 50| 105| 155| -17| -16| 202| 26| 383| 16272| ITF| ITF5|
 Basilicata|2020-05-08 17:00:00| 48| 2| 50| 102| 152| -3| -1| 204| 26| 382| 16777| ITF| ITF5|
 Marche|2020-05-19 17:00:00| 144| 17| 161| 1967| 2128| -187| -3| 3561| 986| 6675| 89985| ITI| ITI3|
 Sardegna|2020-05-25 17:00:00| 49| 3| 52| 179| 231| -14| -2| 994| 129| 1354| 51073| ITG| ITG2|
 Sardegna|2020-06-09 17:00:00| 9| 1| 10| 44| 54| -2| -1| 1176| 131| 1361| 64272| ITG| ITG2|
 Campania|2020-06-12 17:00:00| 61| 2| 63| 283| 346| -264| -229| 3832| 430| 4608| 230551| ITF| ITF3|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT')
dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 1| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 1| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 10| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 10| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 7| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 11| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 9| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 15| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 23| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 22| 27| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 47| 28| 74| 1| 0| 0| 0| 0| 0| 0| 0| 0| 75| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 43| 16| 59| 0| 0| 0| 0| 0| 0| 0| 0| 0| 59| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 27| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 61| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 80| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF| ITF3| Campania|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[94]: [('data_somministrazione', 'string'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_maschile', 'int'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(50)

Il numero di regioni è: 21

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2|Valle d'Aosta / V...|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1|Provincia Autonom...|
 ITH| ITH4|Friuli-Venezia Gi...|
 ITH| ITH3| Veneto|
 ITH| ITH2|Provincia Autonom...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(num_regioni)

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4|Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['sesso_maschile', 'sesso_femminile', 'prima_dose', 'seconda_dose']) \
                             .show()

+-------+------------------+-----------------+-----------------+------------------+
summary| sesso_maschile| sesso_femminile| prima_dose| seconda_dose|
+-------+------------------+-----------------+-----------------+------------------+
 count| 41100| 41100| 41100| 41100|
 mean|192.02817518248176|258.1623114355231|318.7589294403893|131.43155717761556|
 stddev| 544.275878093606| 697.344103413041|961.6079270596197| 517.0437619715813|
 min| 0| 0| 0| 0|
 max| 13942| 18424| 29485| 15344|
+-------+------------------+-----------------+-----------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+
 count| 41100| 41100| 41100| 41100| 41100| 41100| 41100| 41100| 41100| 41100|
 mean| 78.24012165450122| 21.354647201946474| 15.70433090024331|19.427372262773723|64.17396593673966|143.28547445255475| 7.731970802919708| 28.058564476885646| 63.92948905109489| 8.2845498783455|
 stddev| 232.0793778375099| 67.30888509953952| 62.08017204385111| 252.3337700294352|653.1593195919639| 900.0949422540787| 43.341988631678056| 154.4972160494594| 279.10371767509776|37.57116613976922|
 min| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 max| 4661| 2303| 2284| 20017| 28024| 31026| 1214| 3583| 5781| 2008|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+

## Analisi

### Dati covid Regioni

#### Panoramica Casi per Mese

In [0]:
casi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'nuovi_positivi') \
                  .groupby('mese') \
                  .sum('nuovi_positivi') \
                  .withColumnRenamed('sum(nuovi_positivi)', 'casi')

In [0]:
tamponi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'tamponi') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('tamponi') \
                                    .groupby('mese') \
                                    .sum('max(tamponi)') \
                                    .withColumnRenamed('sum(max(tamponi))', 'tamponi')

my_window = Window.partitionBy().orderBy("mese") 

tamponi_mensili = tamponi_mensili.withColumn("prev_value", lag(tamponi_mensili.tamponi) \
                                 .over(my_window)) 
tamponi_mensili = tamponi_mensili.withColumn("diff", when(isnull(tamponi_mensili.tamponi - tamponi_mensili.prev_value), tamponi_mensili.tamponi).otherwise(tamponi_mensili.tamponi - tamponi_mensili.prev_value)) \
                                 .drop('tamponi', 'prev_value') \
                                 .withColumnRenamed('diff', 'tamponi')

In [0]:
panoramica_mensile = casi_mensili.join(tamponi_mensili, on='mese') \
                                 .orderBy('mese', ascending=False)

panoramica_mensile = panoramica_mensile.withColumn('percentuale_tamponi_positivi', panoramica_mensile.casi / panoramica_mensile.tamponi * 100)

panoramica_mensile.show()

+-------+------+-------+----------------------------+
 mese| casi|tamponi|percentuale_tamponi_positivi|
+-------+------+-------+----------------------------+
2021-04|398295|7672073| 5.191491269699858|
2021-03|648200|9769610| 6.634860552263601|
2021-02|372503|7466161| 4.989217350121435|
2021-01|445585|5948709| 7.4904487679595695|
2020-12|508914|4772918| 10.662533904835575|
2020-11|922206|6160638| 14.969326228874348|
2020-10|364607|4450539| 8.192423434554781|
2020-09| 45623|2689063| 1.6966132812805055|
2020-08| 21702|1831746| 1.1847712510358968|
2020-07| 7006|1423003| 0.4923390885331935|
2020-06| 7584|1511371| 0.5017960513996894|
2020-05| 27556|1899522| 1.4506807502097896|
2020-04| 99671|1472249| 6.769982523336745|
2020-03|104664| 488307| 21.434056853577772|
2020-02| 1120| 18661| 6.001821981673007|
+-------+------+-------+----------------------------+

#### Attuali Positivi per Regione

In [0]:
attuali_positivi_regione = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data','denominazione_regione', 'totale_positivi') \
                                             .withColumnRenamed('dati_covid_regionidenominazione_regione', 'regione') \
                                             .withColumnRenamed('totale_positivi', 'attuali_positivi') \
                                             .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd'))

popolazione = popolazione_regioni.groupBy('denominazione_regione') \
                                 .sum('totale_generale')

cond = [popolazione.denominazione_regione == attuali_positivi_regione.regione]
attuali_positivi_regione = attuali_positivi_regione.join(popolazione, cond, how='left') \
                                                   .drop('denominazione_regione') \
                                                   .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                   .orderBy('regione')

attuali_positivi_regione = attuali_positivi_regione.withColumn('positivi_per_100k', (attuali_positivi_regione.attuali_positivi / attuali_positivi_regione.popolazione * 100000).cast('int')) \
                                                   .orderBy('positivi_per_100k', ascending=False)

attuali_positivi_regione.show(21)

+----------+--------------------+----------------+-----------+-----------------+
 data| regione|attuali_positivi|popolazione|positivi_per_100k|
+----------+--------------------+----------------+-----------+-----------------+
2021-04-27| Campania| 91679| 5712143| 1604|
2021-04-27| Puglia| 48661| 3953305| 1230|
2021-04-27| Basilicata| 6059| 553254| 1095|
2021-04-27| Sardegna| 17548| 1611621| 1088|
2021-04-27| Emilia-Romagna| 47803| 4464119| 1070|
2021-04-27| Lazio| 46178| 5755700| 802|
2021-04-27| Calabria| 14675| 1894110| 774|
2021-04-27|Friuli Venezia Gi...| 8044| 1206216| 666|
2021-04-27| Abruzzo| 8614| 1293941| 665|
2021-04-27| Valle d'Aosta| 783| 125034| 626|
2021-04-27| Toscana| 21766| 3692555| 589|
2021-04-27| Sicilia| 26085| 4875290| 535|
2021-04-27| Lombardia| 53051| 10027602| 529|
2021-04-27| Veneto| 22844| 4879133| 468|
2021-04-27| Marche| 6257| 1512672| 413|
2021-04-27| Piemonte| 16764| 4311217| 388|
2021-04-27| Liguria| 5828| 1524826| 382|
2021-04-27| Umbria| 2998| 870165| 344|
2021-04-27| Molise| 606| 300516| 201|
2021-04-27| Trento| 1060| 545425| 194|
2021-04-27| Bolzano| 846| 532644| 158|
+----------+--------------------+----------------+-----------+-----------------+

#### Dati Critici per Mese

In [0]:
decessi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'deceduti') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('deceduti') \
                                    .groupby('mese') \
                                    .sum('max(deceduti)') \
                                    .withColumnRenamed('sum(max(deceduti))', 'deceduti')

my_window = Window.partitionBy().orderBy("mese") 

decessi_mensili = decessi_mensili.withColumn("prev_value", lag(decessi_mensili.deceduti) \
                                 .over(my_window)) 
decessi_mensili = decessi_mensili.withColumn("diff", when(isnull(decessi_mensili.deceduti - decessi_mensili.prev_value), decessi_mensili.deceduti).otherwise(decessi_mensili.deceduti - decessi_mensili.prev_value)) \
                                 .drop('deceduti', 'prev_value') \
                                 .withColumnRenamed('diff', 'deceduti')

In [0]:
terapie_intensive_mensili = dati_covid_regioni.select('data', 'terapia_intensiva') \
                  .groupby('data') \
                  .sum('terapia_intensiva') \
                  .select(date_format('data','yyyy-MM').alias('mese'), 'sum(terapia_intensiva)') \
                  .groupby('mese') \
                  .mean('sum(terapia_intensiva)') \
                  .withColumnRenamed('avg(sum(terapia_intensiva))', 'media_terapia_intensiva')

terapie_intensive_mensili = terapie_intensive_mensili.withColumn('media_terapia_intensiva', terapie_intensive_mensili.media_terapia_intensiva.cast('int'))

In [0]:
dati_critici_mensili = decessi_mensili.join(casi_mensili, on='mese') \
                                      .join(terapie_intensive_mensili, on='mese') \
                                      .orderBy('mese', ascending=False)

dati_critici_mensili.show()

+-------+--------+------+-----------------------+
 mese|deceduti| casi|media_terapia_intensiva|
+-------+--------+------+-----------------------+
2021-04| 10566|398295| 3380|
2021-03| 11647|648200| 3127|
2021-02| 9183|372503| 2128|
2021-01| 14357|445585| 2483|
2020-12| 18583|508914| 3005|
2020-11| 16958|922206| 3231|
2020-10| 2724|364607| 785|
2020-09| 411| 45623| 194|
2020-08| 341| 21702| 58|
2020-07| 311| 7006| 57|
2020-06| 1406| 7584| 207|
2020-05| 5458| 27556| 882|
2020-04| 15539| 99671| 2975|
2020-03| 12399|104664| 1985|
2020-02| 29| 1120| 53|
+-------+--------+------+-----------------------+

### Dati Somministrazioni Vaccini

#### Percentuale Popolazione Vaccinata per Regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 293298| 115704| 1293941| 22.66703041328778| 8.941984217209285|
 Basilicata| 118368| 52122| 553254| 21.394874686852695| 9.420989274365843|
 Bolzano| 131307| 50146| 532644| 24.651925113208822| 9.414543297211646|
 Calabria| 350513| 151246| 1894110| 18.505419431817582| 7.985069504938995|
 Campania| 1160026| 434517| 5712143| 20.308070018555206| 7.6068998972889865|
 Emilia-Romagna| 1042150| 487188| 4464119| 23.34503179686742| 10.913418750709827|
Friuli Venezia Gi...| 272326| 126284| 1206216| 22.576885068677583| 10.46943499340085|
 Lazio| 1265375| 550967| 5755700| 21.984728182497353| 9.572545476657922|
 Liguria| 395814| 165927| 1524826| 25.957978156196184| 10.881700600593117|
 Lombardia| 2173612| 871817| 10027602| 21.67628910680739| 8.694172345491973|
 Marche| 381520| 133754| 1512672| 25.221594635188595| 8.842234139324322|
 Molise| 73176| 35474| 300516| 24.350117797388492| 11.804363162027979|
 Piemonte| 1019991| 424914| 4311217| 23.659003942506256| 9.856010495412317|
 Puglia| 912232| 295267| 3953305| 23.07517381026761| 7.4688646588107925|
 Sardegna| 336397| 141019| 1611621| 20.87320778272311| 8.750134181671745|
 Sicilia| 920261| 415996| 4875290| 18.876025836411785| 8.532743693195686|
 Toscana| 834796| 336153| 3692555| 22.607544098869212| 9.103534002878765|
 Trento| 122803| 41633| 545425| 22.51510290140716| 7.633130127881927|
 Umbria| 205685| 81986| 870165| 23.63747105434027| 9.42189125050996|
 Valle d'Aosta| 31226| 10969| 125034| 24.97400707007694| 8.772813794647856|
 Veneto| 1060116| 478754| 4879133| 21.72754872638233| 9.81227607445831|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 24348| 8101| 2298846| 1.0591401076888143| 0.3523942012644605|
 20-29| 545991| 275841| 6084382| 8.973647611211788| 4.533591086161256|
 30-39| 794083| 367945| 6854632| 11.584618984651547| 5.367830103789671|
 40-49| 1156824| 470721| 8937229| 12.943877794784042| 5.2669680949206965|
 50-59| 1585430| 640708| 9414195| 16.840845128022096| 6.805765123836929|
 60-69| 1940119| 517256| 7364364| 26.344691815885255| 7.023770144984685|
 70-79| 3246926| 433335| 5968373| 54.40219637747171| 7.260521418483731|
 80-89| 3115143| 2195157| 3628160| 85.86013295995767| 60.5033129740695|
 90+| 692128| 492773| 791543| 87.44035384053677| 62.25473537129379|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Regione e Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)

+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 16-19| 622| 198| 46751| 1.3304528245385125| 0.42352035250582876|
 Abruzzo| 20-29| 11529| 4782| 130083| 8.862802979636077| 3.676114480754595|
 Abruzzo| 30-39| 18007| 7866| 150713| 11.947874436843538| 5.219191443339327|
 Abruzzo| 40-49| 27894| 9373| 190060| 14.6764179732716| 4.931600547195623|
 Abruzzo| 50-59| 40956| 11808| 204157| 20.06103146108142| 5.783784048550871|
 Abruzzo| 60-69| 52528| 10452| 167705| 31.321666020691097| 6.232372320443636|
 Abruzzo| 70-79| 58981| 5436| 130572| 45.171246515332534| 4.16322029225255|
 Abruzzo| 80-89| 67699| 53831| 84488| 80.12853896411325| 63.71437363885995|
 Abruzzo| 90+| 15082| 11958| 19515| 77.28414040481681| 61.27594158339738|
 Basilicata| 16-19| 352| 28| 22032| 1.597676107480029| 0.1270878721859114|
 Basilicata| 20-29| 3962| 1180| 61183| 6.475655002206496| 1.9286403085824495|
 Basilicata| 30-39| 6962| 2782| 65204| 10.677259063861113| 4.266609410465615|
 Basilicata| 40-49| 11747| 3833| 78467| 14.970624593778275| 4.884856054137408|
 Basilicata| 50-59| 17205| 4752| 87822| 19.590763134522103| 5.410944865751179|
 Basilicata| 60-69| 19736| 3849| 72817| 27.103560981638903| 5.28585357814796|
 Basilicata| 70-79| 19815| 1081| 51805| 38.24920374481228| 2.086671170736415|
 Basilicata| 80-89| 30994| 28023| 36107| 85.83931093693744| 77.61098955881131|
 Basilicata| 90+| 7595| 6594| 7823| 97.08551706506455| 84.28991435510673|
 Bolzano| 16-19| 338| 128| 23269| 1.452576389187331| 0.5500881000472733|
 Bolzano| 20-29| 5823| 2794| 61094| 9.53121419451992| 4.573280518545193|
 Bolzano| 30-39| 8146| 3477| 63127| 12.90414561122816| 5.5079443027547645|
 Bolzano| 40-49| 12285| 5223| 75664| 16.236255022203423| 6.9028864453372805|
 Bolzano| 50-59| 16244| 6633| 83539| 19.444810208405656| 7.94000406995535|
 Bolzano| 60-69| 30387| 3799| 57041| 53.272207710243514| 6.6601216668711976|
 Bolzano| 70-79| 30881| 4852| 46613| 66.2497586510201| 10.40911333748096|
 Bolzano| 80-89| 22141| 18753| 27108| 81.67699572082043| 69.17884019477644|
 Bolzano| 90+| 5062| 4487| 6165| 82.10867802108677| 72.78183292781833|
 Calabria| 16-19| 710| 252| 76743| 0.9251658131686278| 0.3283687111528087|
 Calabria| 20-29| 12633| 5662| 213889| 5.906334594111899| 2.647167456016906|
 Calabria| 30-39| 22103| 11134| 236668| 9.339243159193469| 4.704480538137814|
 Calabria| 40-49| 32686| 13788| 266417| 12.268736604646099| 5.175345417146804|
 Calabria| 50-59| 47851| 19267| 286804| 16.684216398655526| 6.71782820323287|
 Calabria| 60-69| 73749| 22054| 241215| 30.57396928051738| 9.142880832452377|
 Calabria| 70-79| 70327| 14031| 175208| 40.13914889731062| 8.00819597278663|
 Calabria| 80-89| 74856| 54171| 107720| 69.49127367248423| 50.28871147419235|
 Calabria| 90+| 15598| 10887| 23058| 67.64680371237748| 47.21571688784804|
 Campania| 16-19| 3188| 957| 261694| 1.2182166958355942| 0.36569428416394717|
 Campania| 20-29| 50728| 23007| 693479| 7.315001607835277| 3.3176202884297865|
 Campania| 30-39| 77492| 34151| 715258| 10.834132578733827| 4.774640758998851|
 Campania| 40-49| 112288| 39902| 835597| 13.438056862339142| 4.775268460753209|
 Campania| 50-59| 166192| 55686| 868684| 19.131467829498412| 6.410386285461687|
 Campania| 60-69| 248166| 61327| 670867| 36.991832956457834| 9.14145426738832|
 Campania| 70-79| 272898| 42123| 484380| 56.33965068747677| 8.69627152235848|
 Campania| 80-89| 194814| 154599| 255273| 76.31594410689732| 60.5622216215581|
 Campania| 90+| 34260| 22765| 49044| 69.85563983361878| 46.41750265068102|
 Emilia-Romagna| 16-19| 1985| 691| 160045| 1.2402761723265332| 0.4317535693086319|
 Emilia-Romagna| 20-29| 43759| 23869| 42

# IDEE
- seconda dose (tot) e terapie intensive (medie) per ogni mese
- percentuale seconda dose e terapie intensive di oggi per ogni regione